# Install

# Import

In [1]:
import sys
import os
import gc
import copy
import yaml
import pickle
import random
import joblib 
import shutil
from time import time
import typing as tp
from pathlib import Path
import psutil

import numpy as np
import pandas as pd
import scipy

from tqdm.notebook import tqdm
from sklearn.model_selection import StratifiedGroupKFold, StratifiedKFold
from sklearn.metrics import average_precision_score as APS
import duckdb


import torch
import torchvision
from torch import nn
from torch import optim
from torch.optim import lr_scheduler
from torch.cuda import amp
from torch.nn import BCELoss
from torch.utils.data import Dataset


import timm
from mamba_ssm import Mamba
from transformers import AutoModel, AutoTokenizer

import albumentations as A
from albumentations.pytorch import ToTensorV2


# use one device only
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
con = duckdb.connect()

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

20000_50per_CLM.parquet

In [2]:
class CFG:
    TEST_No = 1
    TEST_NUM = int(1674896/32 * TEST_No)
    TEST_OFFSET = int(TEST_NUM * (TEST_No-1))
    TRAIN_CLM_PATH = Path('../../data/processed/20000_50per_CLM.parquet')
    TEST_CLM_PATH = Path(f'../../data/processed/test_CLM_{TEST_OFFSET}_to_{TEST_NUM}.parquet')
    # TEST_ENC_PATH = Path('../../data/external/test_enc.parquet')
    TRAIN_PATH = Path('../../data/raw/train.parquet')
    TEST_PATH = Path('../../data/raw/test.parquet')
    folds = 2
    max_epoch = 9             # number of max epoch. 1epoch means going around the training dataset.
    batch_size = 8           # batch size. Number of samples passed to the network in one training step
    lr = 1.0e-03              # learning rate. determine step size when updating model's weight
    weight_decay = 1.0e-02    # weight decay. Append regularization term for prevent over fitting
    es_patience = 5           # Timing for early stopping. If there is no improvement within this number of epochs, training will be stopped early.
    seed = 1086               # Random number seed
    deterministic = True      # Enable/disable deterministic behavior. If enabled, the program will produce the same results every time it starts with the same initial conditions and inputs.
    enable_amp = False        # Enable/disable Automatic Mixed Precision. Optimizations for floating point etc.
    device = "cuda" 
    n_classes = 3

In [3]:
# train = pd.read_parquet(CFG.TRAIN_ENC_PATH)
# test = pd.read_parquet(CFG.TEST_ENC_PATH)
# train.head()
# print(len(train))
# print(len(test))

In [4]:
# display(train.head())
# print(len(train))
# print(len(test))

In [5]:
# train = con.query(f"""(SELECT *
#                         FROM parquet_scan('{CFG.TRAIN_PATH}')
#                         LIMIT 60000)""").df()
# test = con.query(f"""(SELECT *
#                         FROM parquet_scan('{CFG.TRAIN_ENC_PATH}')
#                         LIMIT 1674896)""").df()

In [6]:
train_clm = con.query(f"""(SELECT *
                        FROM parquet_scan('{CFG.TRAIN_CLM_PATH}')
                        )""").df()

In [7]:
test = con.query(f"""(SELECT *
                        FROM parquet_scan('{CFG.TEST_PATH}')
                        LIMIT 10000)""").df()

In [8]:
test_clm = con.query(f"""(SELECT *
                        FROM parquet_scan('{CFG.TEST_CLM_PATH}')
                        )""").df()

In [9]:
from sklearn.model_selection import KFold

def split_fold(df:pd.DataFrame):
    # config
    N_FOLDS = 5
    RANDAM_SEED = 42
    df['fold'] = -1

    # object
    skf = KFold(n_splits=N_FOLDS, shuffle=True, random_state=RANDAM_SEED)

    for i, (train_index, test_index) in enumerate(skf.split(df)):
        df.loc[test_index, 'fold'] = i
    
    return df

train_clm = split_fold(train_clm)


In [10]:
train_clm.head()

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,binds,protein_name,bind1,bind2,bind3,fold
0,46212804,0.298898,0.215735,0.239334,0.034285,0.182660,0.425175,-0.200705,0.449326,-0.123180,-0.014747,0.462258,0.081417,0.062271,-0.096787,-0.187001,0.134700,0.340253,0.231603,-0.335292,0.052342,0.381473,0.145885,0.166831,-0.110317,-0.181465,-0.264856,0.143816,-0.313215,-0.080077,0.310048,0.308441,-0.532457,-0.059671,-0.212972,0.105532,-0.054325,0.206103,-0.160668,0.135859,0.036342,-0.213081,-0.002449,0.107417,-0.046394,-0.106699,-0.204243,0.223774,0.186928,0.020975,-0.430665,0.007909,-0.272350,0.282469,-0.083282,-0.085133,0.494959,0.385457,-0.145751,0.043396,0.092727,0.269300,0.071610,0.094146,0.081937,0.250892,0.064236,-0.326001,0.109623,-0.155699,0.295775,-0.015644,0.116024,0.400189,-0.315032,-0.125271,0.039820,0.104457,-0.176178,-0.451672,0.029381,0.153335,0.048843,0.444282,0.314624,0.490374,0.147327,-0.137429,-0.016569,0.419707,-0.206208,-0.330170,0.165429,-0.207598,-0.042514,0.182129,0.211262,0.401372,0.311899,0.247939,0.016801,-0.027303,-0.467440,-0.189535,-0.141048,0.161806,0.308787,-0.139689,0.163796,-0.168635,-0.270844,0.006200,-0.184556,-0.303797,0.063791,-0.315108,-0.442219,0.259693,0.216973,-0.032719,0.046596,0.249516,0.184226,0.023309,-0.457507,0.148761,0.323185,0.083944,0.417990,0.055452,-0.161624,-0.011798,-0.100162,0.045060,-0.027894,0.040675,0.016945,-0.080315,0.048303,0.168409,0.176050,-0.239828,0.158119,0.378190,0.153335,0.417035,0.536460,0.122592,0.052803,0.046599,0.276383,-0.169088,0.179491,-0.278926,-0.013665,0.212649,-0.391150,0.302406,0.237746,0.352361,0.150087,0.019876,-0.370178,0.140923,0.141833,0.203356,0.325828,0.108686,0.447774,-0.225893,-0.530470,-0.009651,0.104518,-0.032129,0.516808,-0.147289,0.161869,0.254155,0.175396,0.079647,-0.282844,0.372475,0.449003,0.171240,-0.101024,-0.154302,-0.186748,0.255352,0.378686,0.093180,0.122055,0.324621,-0.171650,0.114006,-0.067154,0.063546,0.003224,-0.119576,-0.170327,0.187969,0.177499,-0.022536,0.077602,0.101111,-0.071042,0.221414,0.091784,0.128525,0.049797,0.090092,0.206479,0.527189,0.150536,-0.144611,0.500688,0.354053,0.247339,-0.059577,0.147516,0.002164,0.374246,0.297073,0.148663,-0.117241,-0.013537,-0.191889,0.113751,-0.008107,-0.080332,-0.175630,0.159140,-0.392545,-0.545870,-0.063459,0.204859,0.275421,0.365973,0.156539,-0.383238,-0.174160,-0.144050,0.064056,0.180330,-0.306350,-0.120511,0.233948,0.398846,0.216662,-0.289622,0.125582,-0.180842,0.126861,0.060517,0.223539,-0.018413,0.637680,-0.157515,-0.154741,0.091117,-0.107897,0.376983,-0.014590,-0.092441,0.009056,0.155930,-0.021673,-0.087084,-0.442351,0.324968,

In [11]:
display(train_clm.head())
display(train_clm.tail())

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,binds,protein_name,bind1,bind2,bind3,fold
0,46212804,0.298898,0.215735,0.239334,0.034285,0.182660,0.425175,-0.200705,0.449326,-0.123180,-0.014747,0.462258,0.081417,0.062271,-0.096787,-0.187001,0.134700,0.340253,0.231603,-0.335292,0.052342,0.381473,0.145885,0.166831,-0.110317,-0.181465,-0.264856,0.143816,-0.313215,-0.080077,0.310048,0.308441,-0.532457,-0.059671,-0.212972,0.105532,-0.054325,0.206103,-0.160668,0.135859,0.036342,-0.213081,-0.002449,0.107417,-0.046394,-0.106699,-0.204243,0.223774,0.186928,0.020975,-0.430665,0.007909,-0.272350,0.282469,-0.083282,-0.085133,0.494959,0.385457,-0.145751,0.043396,0.092727,0.269300,0.071610,0.094146,0.081937,0.250892,0.064236,-0.326001,0.109623,-0.155699,0.295775,-0.015644,0.116024,0.400189,-0.315032,-0.125271,0.039820,0.104457,-0.176178,-0.451672,0.029381,0.153335,0.048843,0.444282,0.314624,0.490374,0.147327,-0.137429,-0.016569,0.419707,-0.206208,-0.330170,0.165429,-0.207598,-0.042514,0.182129,0.211262,0.401372,0.311899,0.247939,0.016801,-0.027303,-0.467440,-0.189535,-0.141048,0.161806,0.308787,-0.139689,0.163796,-0.168635,-0.270844,0.006200,-0.184556,-0.303797,0.063791,-0.315108,-0.442219,0.259693,0.216973,-0.032719,0.046596,0.249516,0.184226,0.023309,-0.457507,0.148761,0.323185,0.083944,0.417990,0.055452,-0.161624,-0.011798,-0.100162,0.045060,-0.027894,0.040675,0.016945,-0.080315,0.048303,0.168409,0.176050,-0.239828,0.158119,0.378190,0.153335,0.417035,0.536460,0.122592,0.052803,0.046599,0.276383,-0.169088,0.179491,-0.278926,-0.013665,0.212649,-0.391150,0.302406,0.237746,0.352361,0.150087,0.019876,-0.370178,0.140923,0.141833,0.203356,0.325828,0.108686,0.447774,-0.225893,-0.530470,-0.009651,0.104518,-0.032129,0.516808,-0.147289,0.161869,0.254155,0.175396,0.079647,-0.282844,0.372475,0.449003,0.171240,-0.101024,-0.154302,-0.186748,0.255352,0.378686,0.093180,0.122055,0.324621,-0.171650,0.114006,-0.067154,0.063546,0.003224,-0.119576,-0.170327,0.187969,0.177499,-0.022536,0.077602,0.101111,-0.071042,0.221414,0.091784,0.128525,0.049797,0.090092,0.206479,0.527189,0.150536,-0.144611,0.500688,0.354053,0.247339,-0.059577,0.147516,0.002164,0.374246,0.297073,0.148663,-0.117241,-0.013537,-0.191889,0.113751,-0.008107,-0.080332,-0.175630,0.159140,-0.392545,-0.545870,-0.063459,0.204859,0.275421,0.365973,0.156539,-0.383238,-0.174160,-0.144050,0.064056,0.180330,-0.306350,-0.120511,0.233948,0.398846,0.216662,-0.289622,0.125582,-0.180842,0.126861,0.060517,0.223539,-0.018413,0.637680,-0.157515,-0.154741,0.091117,-0.107897,0.376983,-0.014590,-0.092441,0.009056,0.155930,-0.021673,-0.087084,-0.442351,0.324968,

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,binds,protein_name,bind1,bind2,bind3,fold
19995,81985115,0.267760,-0.205158,0.055440,0.164821,-0.094532,0.485431,0.157744,0.006200,0.010068,-0.270387,0.162312,-0.109287,0.045359,0.180453,-0.223929,0.044525,0.247351,0.092405,-0.143986,-0.025025,0.375169,0.127332,0.189733,-0.021936,0.086854,-0.214181,0.169531,-0.240279,-0.400407,0.188539,0.232649,-0.375555,-0.085911,-0.032456,0.308866,-0.146519,0.051496,-0.136750,0.191694,-0.060231,0.101661,0.071745,0.415148,-0.277804,0.107206,0.115251,0.110554,0.190349,0.142135,-0.037027,0.121706,-0.274752,0.283191,-0.145337,-0.259309,0.430909,-0.014229,-0.059993,0.127745,-0.209908,0.277598,-0.162363,-0.013892,0.053965,0.118625,0.209786,-0.471090,0.042351,-0.161719,0.199357,-0.235877,-0.290720,0.268052,-0.300063,0.083030,-0.401781,0.036552,-0.364155,-0.188263,-0.001837,0.007896,0.397756,0.480407,-0.158986,0.339912,0.423772,0.032867,-0.242219,0.030473,-0.104119,-0.527806,-0.128927,-0.050848,0.046085,0.126287,0.328944,0.327661,0.136149,0.126854,-0.087555,0.342546,-0.241172,-0.242538,-0.096584,-0.019571,0.223433,-0.048256,0.088042,0.162187,-0.069042,-0.058806,-0.238028,-0.310510,-0.188983,-0.121560,-0.266385,0.375384,0.169234,-0.048882,-0.152081,0.094466,-0.191125,-0.202785,-0.078845,0.272834,0.292234,-0.275387,0.389165,0.142224,0.122566,0.184430,-0.355373,-0.076050,0.051305,0.174648,-0.043317,0.228175,0.064817,0.120132,0.380199,-0.322808,0.065961,0.506506,0.025589,-0.081243,0.672118,0.419078,0.144714,-0.121433,0.289667,0.243695,-0.112609,-0.240580,0.156492,-0.145750,-0.013503,0.393117,0.329939,-0.051755,0.063881,0.254780,-0.048359,0.323238,-0.012420,0.179485,0.153463,0.226393,0.215787,0.040308,-0.308144,0.131668,0.071805,0.182502,0.348471,-0.426240,0.350583,-0.159535,0.088548,0.226392,-0.212427,0.334456,0.399195,0.361008,-0.149490,0.122159,0.012690,-0.021461,-0.056596,0.114887,-0.121437,0.257064,0.208392,-0.020624,-0.180461,0.020995,-0.004054,-0.314335,-0.183095,0.103433,-0.199091,0.064967,0.097860,0.315047,-0.360973,0.154966,-0.094521,0.279587,-0.024729,0.040306,-0.080189,0.479288,0.003855,0.140855,0.502402,0.107098,0.289421,0.191002,0.061969,-0.036177,0.094025,0.041906,-0.002330,-0.198890,0.251901,-0.266071,0.435110,-0.133821,0.012888,-0.249591,0.164534,-0.320587,0.151441,-0.170706,-0.183306,0.151317,0.325109,-0.140446,-0.507229,0.036000,-0.315991,-0.142227,0.095214,0.024840,-0.034787,0.249153,0.339171,-0.115252,-0.157611,0.241072,0.086354,-0.149471,-0.195313,-0.105829,-0.075470,0.377410,0.116973,0.024813,0.291889,-0.319122,0.377014,-0.142720,-0.017419,0.135625,0.339140,-0.459085,0.013868,-0.

In [12]:
display(test_clm.head())
display(test_clm.tail())

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,protein_name
0,295246830,0.059212,0.268544,0.459275,-0.478332,0.064293,0.066137,-0.147887,0.115527,0.04611,-0.391337,0.092013,-0.490209,0.187342,0.240718,-0.073990,-0.059983,-0.127911,-0.027013,0.112009,0.116732,-0.042179,0.060598,0.155025,0.118495,-0.099980,-0.450256,-0.220516,0.086881,0.253202,-0.138012,-0.08948,0.250520,-0.197369,0.480137,0.277741,0.180488,0.016737,0.351846,0.223353,-0.089910,-0.001178,0.171206,-0.030319,-0.434306,-0.242058,0.14536,-0.075310,-0.217538,-0.225625,-0.132706,0.453836,0.087410,0.054889,0.130223,0.205932,-0.098707,0.078381,0.384029,-0.626169,-0.058532,-0.171075,-0.191866,-0.196223,-0.413641,0.491241,0.022580,-0.291471,0.497175,-0.195436,-0.244227,0.053643,-0.072007,-0.426689,0.147177,0.143980,-0.395837,-0.046297,0.260624,-0.051514,-0.211965,-0.020181,0.427504,-0.041825,-0.270080,-0.079692,0.186883,-0.038735,-0.109503,-0.044395,-0.211253,0.220032,0.349634,0.308177,0.133785,-0.180837,0.133978,0.190345,0.299555,0.286484,0.280494,0.327245,-0.004544,0.231393,0.275506,0.428730,0.347076,0.32989,-0.392831,0.244522,-0.000133,-0.174336,0.269797,0.185783,-0.081361,-0.312756,0.051407,0.302243,0.120902,-0.162387,-0.137473,0.398268,0.023792,0.232194,0.297438,0.074692,-0.102134,-0.131429,-0.209996,0.235439,-0.296839,-0.198691,0.242217,0.252454,-0.148691,0.097730,-0.028268,-0.152825,-0.136804,-0.242104,-0.167165,-0.127469,-0.147285,0.117006,-0.044710,-0.006301,0.253113,0.318814,-0.018872,0.090891,-0.159270,0.151642,0.422762,-0.211887,0.06089,-0.247174,0.285255,-0.069700,0.222489,-0.189423,-0.226176,0.021991,0.342161,0.025141,-0.079073,-0.166883,0.066095,0.107221,0.095281,-0.228454,-0.177098,-0.272340,0.228061,0.031688,0.173032,-0.205826,0.112604,0.225269,-0.317386,-0.219432,0.038121,0.130454,-0.136679,0.121882,0.110887,-0.223436,0.339159,0.179406,0.375215,-0.378125,-0.288394,-0.073626,0.190888,0.119932,0.000022,-0.416229,0.151107,0.232573,-0.036283,-0.230874,0.196991,0.069661,-0.16091,-0.063775,0.086544,-0.21288,0.361036,-0.041675,0.005949,0.291126,0.116930,0.257231,0.203121,0.101177,-0.284215,-0.064979,-0.571144,-0.005481,-0.225907,-0.136872,-0.411398,0.014252,-0.168130,-0.145796,0.262412,-0.321651,-0.496312,0.306908,-0.112732,0.108853,-0.079686,-0.083129,0.128327,-0.079738,0.065211,-0.199437,-0.161753,-0.319671,-0.111416,0.086411,0.321680,-0.238752,-0.355464,-0.025144,0.204006,-0.307873,0.129757,-0.247858,-0.159178,0.412053,-0.170795,0.071709,-0.503352,-0.129929,-0.282270,-0.042521,0.038287,-0.178111,0.179725,-0.157668,0.021689,-0.022641,0.042579,0.341073,0.406713,0.027472,-0.114482,0.103727,0.281378,0.07386

,id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,protein_name
52335,295299165,-0.372445,0.089123,0.418904,-0.146498,-0.177251,-0.063790,0.174884,-0.163991,-0.124751,-0.325546,-0.149637,-0.282449,-0.086926,0.024559,0.201457,-0.195212,0.127227,-0.074495,-0.133644,0.074666,0.119740,0.016350,0.117549,0.181753,0.283407,-0.519334,-0.263275,0.337384,-0.183502,-0.429861,0.404957,0.068461,-0.280357,0.267928,0.249911,0.270621,0.001820,0.379375,0.006505,0.035273,-0.235901,-0.342318,0.263929,-0.272717,0.242168,-0.042507,0.247759,-0.067091,-0.433868,0.144421,0.070918,-0.047270,-0.167043,0.083137,0.205522,-0.026024,-0.257766,-0.090201,-0.353702,0.000230,0.258851,-0.106479,-0.383016,-0.182588,0.231202,0.131400,-0.211584,0.352473,-0.131599,-0.283094,0.292397,-0.107225,-0.523863,-0.061351,0.033532,0.092375,-0.239060,-0.012586,0.064479,-0.200928,0.210401,-0.021280,-0.483799,-0.311418,-0.137513,-0.230957,0.250006,-0.322533,-0.034926,0.002235,0.293057,0.036558,0.220063,-0.304656,-0.276558,-0.334049,-0.182484,0.166385,0.429743,0.310387,0.382306,0.176225,0.127668,0.059181,0.368785,0.367715,0.107728,-0.489164,0.133242,-0.038010,-0.161766,0.141541,0.392442,-0.022184,-0.142277,0.305699,0.052402,0.301233,-0.043504,-0.094255,0.326555,-0.353080,0.125042,0.136899,0.351833,0.118053,0.072015,-0.238604,-0.207195,-0.127761,-0.359877,0.311718,0.163081,-0.105712,-0.198508,0.210891,-0.103324,-0.069459,-0.348373,-0.076111,-0.050813,0.030590,0.107551,-0.100178,-0.290545,0.060321,0.068098,0.029739,0.242438,-0.156996,0.222949,0.283157,-0.332238,-0.066042,-0.219336,-0.285994,0.023910,0.071221,0.167493,-0.169378,0.278272,0.190303,-0.238058,0.077009,-0.180238,0.225606,-0.250502,-0.037546,-0.041627,0.134651,0.145901,0.047593,0.262138,0.228806,0.243071,0.094799,0.115821,-0.281426,-0.112634,0.065977,0.129159,-0.152682,0.235771,-0.337645,-0.050427,0.152164,0.137437,0.257991,-0.230863,0.046560,0.220159,0.155795,-0.157479,0.104969,-0.423726,0.010847,0.200856,0.036934,-0.096868,-0.249030,0.037687,-0.126697,0.189043,0.212565,-0.207688,0.294509,0.013881,-0.172653,0.136507,-0.145554,0.387191,0.046742,-0.256215,-0.154845,0.000249,-0.195086,-0.015970,-0.398631,0.212400,-0.391188,-0.062532,-0.024880,-0.037434,0.092666,-0.256986,-0.277054,0.078435,0.118048,-0.001850,-0.130171,-0.399396,0.111800,0.014490,0.162363,-0.216092,-0.070949,-0.335243,-0.036356,0.014307,0.013512,-0.175329,-0.021197,0.281631,0.217629,-0.365577,0.377387,-0.160751,-0.455030,0.342863,-0.097115,0.190776,-0.482534,-0.353216,-0.041615,-0.136838,-0.145850,-0.340617,0.305948,-0.135634,0.082722,-0.068570,0.284959,0.290537,0.491608,0.155272,-0.010980,0.202743,0.0180

# Dataset

In [13]:
class EXDataset(Dataset):
    def __init__(
        self,
        train: pd.DataFrame,
        label: pd.DataFrame = pd.DataFrame(),
        is_test: bool = False,
        transform = None
    ):
        self.train = train
        self.label = label
        self.is_test = is_test
        self.transform = transform
        
    def __len__(self):
        # return total num of data
        return len(self.train)
    
    def __getitem__(self, index:int):
        # return data and target assosiated with index
        X = self.train.iloc[index]
        X = self._apply_transform(X)
        
        if self.is_test:
            y = np.argmax(np.zeros(CFG.n_classes))
            # y = [0, 0, 0]
        else:
            y = np.argmax(self.label.iloc[index].values)
            # y = self.label.iloc[index].values
        return X, y
    
    def _apply_transform(self, X):
        if self.transform:
            X = self.transform(X)
        return X

# Model

In [14]:
class MambaModel(nn.Module):
    def __init__(self, 
                 dim_model=384, # Model dimension d_model (embedding size)
                 d_state=16, # SSM state expansion factor
                 d_conv=4, # Local convolution width
                 expand=2, # Block expansion factor
                 output = 3, # number of classes (or output number simply)
                #  is_test=False
                 ):
        super().__init__()
        self.model = Mamba(
            d_model=dim_model,  
            d_state=d_state,  
            d_conv=d_conv,    
            expand=expand,    
        ).to("cuda")
        # mamba pass trought input size as is.
        self.output = nn.Linear(dim_model, output)
        self.softmax = nn.Softmax(dim=-1)
        # self.is_test = is_test

    def forward(self, x):
        # Add the length dimension if input has only 2 dimensions
        if len(x.shape) == 2:
            x = x.unsqueeze(1)
            
        x = self.model(x)
        x = self.output(x)
        x = x.squeeze()
        # if self.is_test:
            # x = self.softmax(x)
        return x

##### set seeds

In [15]:
def set_random_seed(seed: int = 42, deterministic: bool = False):
    """Set seeds"""
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)  # type: ignore
    torch.backends.cudnn.deterministic = deterministic  # type: ignore

##### move tensors to device

In [16]:
def to_device(
    tensors: tp.Union[tp.Tuple[torch.Tensor], tp.Dict[str, torch.Tensor]],
    device: torch.device, *args, **kwargs
):
    if isinstance(tensors, tuple):
        return (t.to(device, *args, **kwargs) for t in tensors)
    elif isinstance(tensors, dict):
        return {
            k: t.to(device, *args, **kwargs) for k, t in tensors.items()}
    else:
        return tensors.to(device, *args, **kwargs)

##### transform values to tensor

In [17]:
def to_tensor(x):
    return torch.tensor(x.values, dtype=torch.float32)

# Train

In [18]:
def train_one_fold(CFG,
                   val_fold: int,
                   train: pd.DataFrame,
                   output_path
                   ):
    feature_columns = [str(i) for i in range(384)]
    label_columns = ['bind1', 'bind2', 'bind3']

    set_random_seed(CFG.seed, deterministic=CFG.deterministic)
    device = torch.device(CFG.device)
    train_dataset = EXDataset(train = train[feature_columns][train['fold']!=val_fold].reset_index(drop=True), 
                              label = train[label_columns][train['fold']!=val_fold].reset_index(drop=True), 
                              transform = to_tensor)
    val_dataset = EXDataset(train = train[feature_columns][train['fold']==val_fold].reset_index(drop=True), 
                            label = train[label_columns][train['fold']==val_fold].reset_index(drop=True), 
                            transform = to_tensor)

    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=True, drop_last=True)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)

    model = MambaModel()
    model.to(device)

    optimizer = optim.AdamW(params=model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
    scheduler = lr_scheduler.OneCycleLR(
        optimizer=optimizer, epochs=CFG.max_epoch,
        pct_start=0.0, steps_per_epoch=len(train_loader),
        max_lr=CFG.lr, div_factor=25, final_div_factor=4.0e-01
    )
    loss_func = nn.CrossEntropyLoss()
    loss_func.to(device)
    loss_func_val = nn.CrossEntropyLoss()

    use_amp = CFG.enable_amp
    scaler = amp.GradScaler(enabled=use_amp)

    best_val_loss = 1.0e+09
    best_epoch = 0
    train_loss = 0
    val_loss = 0
    
    for epoch in range(1, CFG.max_epoch + 1):
        epoch_start = time()
        model.train()
        for batch in train_loader:
            
            x, t = batch
            # print(x)
            # print(t)
            x = to_device(x, device)
            t = to_device(t, device).long()
            # sys.exit()
                
            optimizer.zero_grad()
            with amp.autocast(use_amp):
                y = model(x)
                loss = loss_func(y, t)
            
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item()
            scheduler.step()
            
        train_loss /= len(train_loader)
            
        model.eval()
        for batch in val_loader:
            x, t = batch
            x = to_device(x, device)
            with torch.no_grad(), amp.autocast(use_amp):
                y = model(x)
#                 y = torch.sigmoid(y)
            y = y.detach().cpu().to(torch.float32)
            loss = loss_func_val(y, t)
            val_loss += loss.item()
        val_loss /= len(val_loader)
        
        if val_loss < best_val_loss:
            best_epoch = epoch
            best_val_loss = val_loss
            # print("save model")
            torch.save(model.state_dict(), str(output_path / f'snapshot_epoch_{epoch}.pth'))
        
        elapsed_time = time() - epoch_start
        print(
            f"[epoch {epoch}] train loss: {train_loss: .6f}, val loss: {val_loss: .6f}, elapsed_time: {elapsed_time: .3f}")
        
        if epoch - best_epoch > CFG.es_patience:
            print("Early Stopping!")
            break
            
        train_loss = 0
        val_loss = 0
            
    return val_fold, best_epoch, best_val_loss

Do the above function actually, and save the best model of each epoch.



In [19]:
score_list = []
for fold_id in range(CFG.folds):
    output_path = Path(f"fold{fold_id}")
    output_path.mkdir(exist_ok=True)
    print(f"[fold{fold_id}]")
    score_list.append(train_one_fold(CFG, fold_id, train_clm, output_path))

[fold0]
[epoch 1] train loss:  0.710686, val loss:  0.639821, elapsed_time:  6.895
[epoch 2] train loss:  0.622275, val loss:  0.616683, elapsed_time:  8.243
[epoch 3] train loss:  0.589275, val loss:  0.613938, elapsed_time:  7.360
[epoch 4] train loss:  0.561334, val loss:  0.586322, elapsed_time:  7.066
[epoch 5] train loss:  0.533720, val loss:  0.557963, elapsed_time:  7.064
[epoch 6] train loss:  0.506600, val loss:  0.550037, elapsed_time:  6.984
[epoch 7] train loss:  0.482904, val loss:  0.552444, elapsed_time:  7.560
[epoch 8] train loss:  0.459771, val loss:  0.549161, elapsed_time:  7.284
[epoch 9] train loss:  0.444606, val loss:  0.547024, elapsed_time:  7.081
[fold1]
[epoch 1] train loss:  0.715190, val loss:  0.641046, elapsed_time:  7.489
[epoch 2] train loss:  0.621275, val loss:  0.623827, elapsed_time:  7.047
[epoch 3] train loss:  0.588653, val loss:  0.599075, elapsed_time:  6.971
[epoch 4] train loss:  0.557471, val loss:  0.592497, elapsed_time:  7.440
[epoch 5]

Check the result.

In [20]:
print(score_list)

[(0, 9, 0.5470238849818706), (1, 9, 0.5417573406845331)]


Delete models without best:

In [21]:
# select the best model and delete others
best_log_list = []
for (fold_id, best_epoch, _) in score_list:
    
    # select the best model
    exp_dir_path = Path(f"fold{fold_id}")
    best_model_path = exp_dir_path / f"snapshot_epoch_{best_epoch}.pth"
    # copy to new place
    copy_to = f"./best_model_fold{fold_id}.pth"
    shutil.copy(best_model_path, copy_to)
    
    for p in exp_dir_path.glob("*.pth"):
        # delete
        p.unlink()

# Infer

In [22]:
def run_inference_loop(model, loader, device):
    model.to(device)
    model.eval()
    pred_list = []
    with torch.no_grad():
        for batch in tqdm(loader):
            x = to_device(batch[0], device)
            y = model(x)
            pred_list.append(y.detach().cpu().numpy())
    
    # concatenate to vertical (to df that like long scroll)
    pred_arr = np.vstack(pred_list)
    del pred_list
    return pred_arr

Do inference actually.

In [25]:
def inference(test):
    test_pred_arr = np.zeros((CFG.folds, len(test), CFG.n_classes))
    score_list = []

    for fold_id in range(CFG.folds):
        print(f"\n[fold {fold_id}]")
        device = torch.device(CFG.device)
        
        feature_columns = [str(i) for i in range(384)]
        test_dataset = EXDataset(test[feature_columns],
                                 transform = to_tensor,
                                 is_test = True)
    
        # test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=CFG.batch_size, num_workers=4, shuffle=False, drop_last=False)
        test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, num_workers=4, shuffle=False, drop_last=False)

        # get model
        model_path = f"./best_model_fold{fold_id}.pth"
        model = MambaModel()
        model.load_state_dict(torch.load(model_path, map_location=device))

        # inference
        test_pred = run_inference_loop(model, test_loader, device)
        test_pred_arr[fold_id] = test_pred

        del model, test_loader
        torch.cuda.empty_cache()
        gc.collect()
    return test_pred_arr

test_preds_arr = inference(test_clm)


[fold 0]


  0%|          | 0/1636 [00:00<?, ?it/s]


[fold 1]


  0%|          | 0/1636 [00:00<?, ?it/s]

mean each fold's predict.

In [26]:
test_pred = test_preds_arr.mean(axis=0)
test_pred = pd.DataFrame(test_pred)
# test_pred = pd.concat([test_clm[['id', 'protein_name']], test_pred], axis=1)
test_pred = pd.concat([test_clm['id'], test_pred], axis=1)

In [27]:
display(test_pred.head(100))

,id,0,1,2
0,295246830,-1.822291,1.019120,1.441357
1,295246831,-1.822291,1.019120,1.441357
2,295246832,-1.822291,1.019120,1.441357
3,295246833,-1.739148,1.164691,1.236203
4,295246834,-1.739148,1.164691,1.236203
5,295246835,-1.739148,1.164691,1.236203
6,295246836,0.705985,0.336310,-1.147296
7,295246837,0.705985,0.336310,-1.147296
8,295246838,0.705985,0.336310,-1.147296
9,295246839,-1.067443,0.960176,0.526411


# Submission

In [ ]:
# soon...